In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import re
import json

In [5]:
# Format:
# Bowler, batsman 
# Runs in numbers/words, out else 0. If wide/no ball then ignore entry
# out
def to_corrrect_format(filepath, innings_num): 
    df = pd.read_csv(filepath, sep='delimiter',header=None, engine="python") #, names=["Players", "Outcome"]
    df = df[::-1]
    df.reset_index(drop=True, inplace=True)

    json_path = filepath.split("texts")
    f = open(json_path[0]+'match_info.json')
    json_f = json.load(f)
    bat_suf = str(json_f['First Innings'])
    bowl_suf = str(json_f['Second Innings'])
    if innings_num == 2:
        bat_suf = str(json_f['Second Innings'])
        bowl_suf = str(json_f['First Innings'])
    f.close()
    
    numbers_dict = {"zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6}
    data = []
    wkt_taken = 0
    num_over = 0
    prev_bowler = ''
    for _, row in df.iterrows():
        line = row.to_string(index=False)
        list = line.split(",")

        bowler, batsman = list[0].split(" to ") 
        batsman = batsman + '_' + bat_suf   
        bowler = bowler + '_' + bowl_suf
        if prev_bowler!=bowler: # what about case senstivity?
            prev_bowler=bowler
            num_over+=1

        if num_over>20:
            # print("Error")
            continue    

        find_runs = r'(zero|one|two|three|four|five|six|0|1|2|3|4|5|6)'
        find_extra = r'(wide|no ball)'
        outcome = re.findall(find_runs, list[1], flags=re.IGNORECASE)
        if re.findall("out", list[-1], flags=re.IGNORECASE):
            
            data.append([bowler, batsman, wkt_taken, num_over, 7])
            wkt_taken += 1
        elif outcome:
            # print(outcome)
            runs = 0
            if outcome[0].isdigit():
                runs = int(outcome[0])
            else:
                num = numbers_dict.get(outcome[0].lower(), None)
                if num is not None:
                    runs = num
            data.append([bowler, batsman, wkt_taken, num_over, runs])
        elif re.findall(find_extra, list[1], flags=re.IGNORECASE):
            continue
        else:
            data.append([bowler, batsman, wkt_taken, num_over, 0])
            # print("yay!")
        # print(list)
    # print(data)
    new_path = filepath.split(".")
    with open(new_path[0]+'.pkl', 'wb') as file:
        pickle.dump(data, file)

In [6]:
# current issues:
#   - Same player last names
#   - Dismissal on extras leads to >6 balls in an over
#   - counting byes as runs

In [8]:
# df = pd.read_csv("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv", sep='delimiter',header=None) #, names=["Players", "Outcome"]
# df.head()

to_corrrect_format("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv", 1)

In [12]:
rootdir = 'T20_matches'
cnt=0
for subdir, dirs, files in os.walk(rootdir):
    # if(cnt>=10):
    #     break
    try: 
        for file in files:
            # if(cnt>=10):
            #     break
            if(file[-1]=='l'):
                continue
            if file.endswith('1.csv'):
                # print(os.path.join(subdir,file ))
                to_corrrect_format(os.path.join(subdir, file), 1)
            elif file.endswith('2.csv'):
                to_corrrect_format(os.path.join(subdir, file), 2)
            # cnt+=1
    except: 
        pass

In [9]:
with open('T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.pkl', 'rb') as f:
    data = pickle.load(f)

In [10]:
df = pd.DataFrame(data)

In [11]:
df

,0,1,2,3,4
0,Mortaza_Bangladesh,Warner_Australia,0,1,0
1,Mortaza_Bangladesh,Warner_Australia,0,1,0
2,Mortaza_Bangladesh,Warner_Australia,0,1,1
3,Mortaza_Bangladesh,Watson_Australia,0,1,0
4,Mortaza_Bangladesh,Watson_Australia,0,1,1
...,...,...,...,...,...
119,Shafiul_Bangladesh,Hussey_Australia,6,20,1
120,Shafiul_Bangladesh,Smith_Australia,6,20,1
121,Shafiul_Bangladesh,Hussey_Australia,6,20,2
122,Shafiul_Bangladesh,Hussey_Australia,6,20,7


In [13]:
data1=[]
rootdir = 'T20_matches'
# cnt=0
for subdir, dirs, files in os.walk(rootdir):
    # if(cnt>=20):
    #     break
    try: 
        for file in files:
            if(file[-1]=='v'):
                continue
            # if(cnt>=20):
            #     break
            if file.endswith('1.pkl'):
                print(os.path.join(subdir,file ))
                with open(os.path.join(subdir,file), 'rb') as f:
                    data1+= (pickle.load(f))
            # cnt+=1
    except: 
        pass

T20_matches\australia-vs-bangladesh-11th-match-group-a-412688\textsInnings1.pkl
T20_matches\australia-vs-bangladesh-14th-match-group-f-287866\textsInnings1.pkl
T20_matches\australia-vs-england-1st-t20i-446960\textsInnings1.pkl
T20_matches\australia-vs-england-2nd-t20i-446961\textsInnings1.pkl
T20_matches\australia-vs-england-9th-match-group-b-287861\textsInnings1.pkl
T20_matches\australia-vs-england-final-412703\textsInnings1.pkl
T20_matches\australia-vs-england-only-t20i-249227\textsInnings1.pkl
T20_matches\australia-vs-india-15th-match-group-f-412691\textsInnings1.pkl
T20_matches\australia-vs-india-16th-match-group-2-533287\textsInnings1.pkl
T20_matches\australia-vs-india-1st-t20i-518954\textsInnings1.pkl
T20_matches\australia-vs-india-2nd-semi-final-287878\textsInnings1.pkl
T20_matches\australia-vs-india-2nd-t20i-518955\textsInnings1.pkl
T20_matches\australia-vs-india-only-t20i-291356\textsInnings1.pkl
T20_matches\australia-vs-new-zealand-only-t20i-291343\textsInnings1.pkl
T20_match

In [14]:
data1

[['Mortaza_Bangladesh', 'Warner_Australia', 0, 1, 0],
 ['Mortaza_Bangladesh', 'Warner_Australia', 0, 1, 0],
 ['Mortaza_Bangladesh', 'Warner_Australia', 0, 1, 1],
 ['Mortaza_Bangladesh', 'Watson_Australia', 0, 1, 0],
 ['Mortaza_Bangladesh', 'Watson_Australia', 0, 1, 1],
 ['Mortaza_Bangladesh', 'Warner_Australia', 0, 1, 0],
 ['Shafiul_Bangladesh', 'Watson_Australia', 0, 2, 0],
 ['Shafiul_Bangladesh', 'Watson_Australia', 0, 2, 1],
 ['Shafiul_Bangladesh', 'Watson_Australia', 0, 2, 0],
 ['Shafiul_Bangladesh', 'Watson_Australia', 0, 2, 2],
 ['Shafiul_Bangladesh', 'Watson_Australia', 0, 2, 1],
 ['Shafiul_Bangladesh', 'Warner_Australia', 0, 2, 6],
 ['Shafiul_Bangladesh', 'Warner_Australia', 0, 2, 4],
 ['Mortaza_Bangladesh', 'Watson_Australia', 0, 3, 7],
 ['Mortaza_Bangladesh', 'Warner_Australia', 1, 3, 0],
 ['Mortaza_Bangladesh', 'Warner_Australia', 1, 3, 4],
 ['Mortaza_Bangladesh', 'Warner_Australia', 1, 3, 1],
 ['Mortaza_Bangladesh', 'Clarke_Australia', 1, 3, 0],
 ['Mortaza_Bangladesh', 'Cla

In [15]:
df1= pd.DataFrame(data1)

In [16]:
df1

,0,1,2,3,4
0,Mortaza_Bangladesh,Warner_Australia,0,1,0
1,Mortaza_Bangladesh,Warner_Australia,0,1,0
2,Mortaza_Bangladesh,Warner_Australia,0,1,1
3,Mortaza_Bangladesh,Watson_Australia,0,1,0
4,Mortaza_Bangladesh,Watson_Australia,0,1,1
...,...,...,...,...,...
30546,Vitori_Zimbabwe,Shehzad_Pakistan,1,20,2
30547,Vitori_Zimbabwe,Shehzad_Pakistan,1,20,2
30548,Vitori_Zimbabwe,Shehzad_Pakistan,1,20,1
30549,Vitori_Zimbabwe,Hafeez_Pakistan,1,20,1


In [17]:
df1=df1.rename(columns={0:"bowler",1:"batsman",2:"wicket",3:"over",4:"outcome"})

In [18]:
df1.columns

Index(['bowler', 'batsman', 'wicket', 'over', 'outcome'], dtype='object')

In [19]:
df1[df1['over']==21]

,bowler,batsman,wicket,over,outcome


In [20]:
df1.to_pickle("./df1.pkl")

In [42]:
#seedha load df1 from df.pkl

In [21]:
np.where(df1['wicket']==10)

(array([], dtype=int64),)

In [22]:
batsmen_dict = {batsman: index for index, batsman in enumerate(df1['batsman'].unique())}
print("Dictionary of batsmen names:", batsmen_dict)

Dictionary of batsmen names: {'Warner_Australia': 0, 'Watson_Australia': 1, 'Clarke_Australia': 2, 'Haddin_Australia': 3, 'Hussey_Australia': 4, 'White_Australia': 5, 'Smith_Australia': 6, 'Harris_Australia': 7, 'Tamim_Bangladesh': 8, 'Nazimuddin_Bangladesh': 9, 'Aftab_Bangladesh': 10, 'Ashraful_Bangladesh': 11, 'Shakib_Bangladesh': 12, 'Mortaza_Bangladesh': 13, 'Kapali_Bangladesh': 14, 'Farhad_Bangladesh': 15, 'Mushfiqur_Bangladesh': 16, 'Finch_Australia': 17, 'Paine_Australia': 18, "O'Keefe_Australia": 19, 'Johnson_Australia': 20, 'Maddy_England': 21, 'Prior_England': 22, 'Wright_England': 23, 'Pietersen_England': 24, 'PD Collingwood_England': 25, 'Flintoff_England': 26, 'Shah_England': 27, 'AD Mascarenhas_England': 28, 'Schofield_England': 29, 'Broad_England': 30, 'Kirtley_England': 31, 'Gilchrist_Australia': 32, 'Hayden_Australia': 33, 'Ponting_Australia': 34, 'Symonds_Australia': 35, 'Gambhir_India': 36, 'Pathan_India': 37, 'Kohli_India': 38, 'Yuvraj_India': 39, 'Rohit Sharma_Indi

In [23]:
# Create the four-dimensional matrix X
num_batsmen = len(batsmen_dict)
max_over = 20 
max_wicket = 10 
max_outcome = 8

X = np.zeros((num_batsmen, max_over, max_wicket, max_outcome), dtype=int)

# Populate the matrix X based on DataFrame values
for index, row in df1.iterrows():
    batsman_idx = batsmen_dict[row['batsman']]
    over_idx = row['over']-1
    wicket_idx = row['wicket']
    outcome_idx = row['outcome']
    
    X[batsman_idx, over_idx, wicket_idx, outcome_idx] += 1

print("Four-dimensional matrix X:")
print(X)

Four-dimensional matrix X:
[[[[41 20  6 ...  0  0  1]
   [ 0  1  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[23 18  3 ...  0  2  3]
   [ 2  3  0 ...  0  0  0]
   [ 5  1  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 7 10  4 ...  0  4  2]
   [ 3  5  1 ...  0  0  0]
   [ 5  1  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  ...

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0 

In [25]:
X[batsmen_dict['Kohli_India']][14][2]

array([2, 2, 0, 0, 0, 0, 0, 1])

In [26]:
np.save("X_mat.npy", X)